# Upload files to Microsoft Azure Storage Account

https://learn.microsoft.com/en-us/azure/storage/blobs/storage-blob-upload-python

In [20]:
from dotenv import load_dotenv

load_dotenv()

True

In [21]:
job_id = "_00000000001"
job_data_file_path = "../outputs/" + job_id + "_data.json"

In [22]:
import json
from sigmaepsilon.deepdict import DeepDict, asciiprint

with open(job_data_file_path, "r") as json_file:
    job_data = json.load(json_file)
job_data : DeepDict = DeepDict.wrap(job_data)
job_data.lock()
asciiprint(job_data)

DeepDict
 +-- _meta
 +-- load
 +-- geometry
 +-- material
 +-- calculation
 |   +-- azure
 |       +-- storage
 +-- output


## Input data

In [23]:
job_id = job_data["_meta","job_id"]

files_to_send = {
    job_id + "_data":job_data["_meta"]["job_data_file_path"],
    job_id + "_res2d":job_data["output"]["absolute_file_path_res2d"],
    job_id + "_res3d":job_data["output"]["absolute_file_path_res3d"],
}
container_name = job_data["calculation"]["azure"]["storage"]["container_name"]

In [24]:
files_to_send

{'_00000000001_data': 'd:\\Projects\\SigmaEpsilon\\sigmaepsilon.solid.fourier\\apps\\plate_calculator_papermill_azure_pyvista\\outputs\\_00000000001_data.json',
 '_00000000001_res2d': 'd:\\Projects\\SigmaEpsilon\\sigmaepsilon.solid.fourier\\apps\\plate_calculator_papermill_azure_pyvista\\tasks\\_00000000001_res2d.parquet',
 '_00000000001_res3d': 'd:\\Projects\\SigmaEpsilon\\sigmaepsilon.solid.fourier\\apps\\plate_calculator_papermill_azure_pyvista\\tasks\\_00000000001_res3d.parquet'}

In [25]:
container_name

'navier'

## Create a client object

In [26]:
import os
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

storage_account_name = os.environ.get("AZURE_STORAGE_ACCOUNT_NAME")
account_url = f"https://{storage_account_name}.blob.core.windows.net"
credential = DefaultAzureCredential()
blob_service_client = BlobServiceClient(account_url, credential=credential)

## Upload a block blob from a local file path

Create a helper function

In [27]:
import os
from azure.storage.blob import BlobClient
from functools import partial


def upload_blob_file(
    blob_service_client: BlobServiceClient,
    container_name: str,
    file_path: str,
    file_name: str,
) -> BlobClient:
    container_client = blob_service_client.get_container_client(
        container=container_name
    )
    with open(file=file_path, mode="rb") as data:
        blob_client = container_client.upload_blob(
            name=file_name, data=data, overwrite=True
        )
    return blob_client


folder_path = os.getcwd()
uploader = partial(upload_blob_file, blob_service_client, container_name)

In [28]:
folder_path = os.getcwd()
for file_name, file_path in files_to_send.items():
    uploader(file_name=file_name, file_path=file_path)